# # Classifiers introduction

In the following program we introduce the basic steps of classification of a dataset in a matrix

Import the package for learning and modeling trees

In [ ]:
from sklearn import tree 

Define the matrix containing the data (one example per row)
and the vector containing the corresponding target value

In [ ]:
X = [[0, 0, 0], [1, 1, 1], [0, 1, 0], [0, 0, 1], [1, 1, 0], [1, 0, 1]] #TRAINING SET (EXAMPLES)
Y = [1, 0, 0, 0, 1, 1] #TRAINING SET (LABELS)

Declare the classification model you want to use and then fit the model to the data

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

Predict the target value (and print it) for the passed data, using the fitted model currently in clf

In [ ]:
print(clf.predict([[0, 1, 1]])) #esempio ignoto

In [ ]:
print(clf.predict([[1, 0, 1],[0, 0, 1]])) #due esempi noti

Visualizziamo come è fatto l'albero che è stato creato con il fit

In [1]:
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None)  #crea uno script in DOT, linguaggio usato per descrivere grafici
graph = graphviz.Source(dot_data)  
graph

ModuleNotFoundError: No module named 'graphviz'

In the following we start using a dataset (from UCI Machine Learning repository)

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

# Declare the type of prediction model and the working criteria for the model induction algorithm

In [ ]:
clf = tree.DecisionTreeClassifier(criterion="entropy",random_state=300,min_samples_leaf=5,class_weight={0:1,1:1,2:1}) 
#random_state: per fissare la randomizzazione e quindi ottenere un comportamento deterministico per la randomizzazione
#min_samples_leaf: una foglia dovrà contenere ALMENO 5 esempi per esistere (cosi si evita overfitting)
#class_weight: indice:peso

# Split the dataset in training and test set

In [ ]:
# random permutation on indexes: DIVIDERÀ TRAINING E TEST SET
import numpy as np
np.random.seed(0)
indices = np.random.permutation(len(iris.data)) #INDICI DA 0 A 149 (len(iris.data)) PERMUTATI RANDOM

# We now decide to keep the last 10 indices for test set, the remaining for the training set
indices_training=indices[:-10] #INDICI TRAINING SET: PRIMI 140 ELEMENTI DEL VETTORE RANDOM
indices_test=indices[-10:] #INDICI TEST SET: ULTIMI 10 ELEMENTI DEL VETTORE RANDOM

iris_X_train = iris.data[indices_training]
iris_y_train = iris.target[indices_training]
iris_X_test  = iris.data[indices_test]
iris_y_test  = iris.target[indices_test]

# Fit the learning model on training set

In [ ]:
# fit the model to the training data
clf = clf.fit(iris_X_train, iris_y_train)

# Obtain predictions

In [ ]:
# apply fitted model "clf" to the test set 
predicted_y_test = clf.predict(iris_X_test)

# print the predictions (class numbers associated to classes names in target names)
print("Predictions:")
print(predicted_y_test)
print("True classes:")
print(iris_y_test) 
print(iris.target_names)

#QUINDI C'è STATO UN SOLO ERRORE, SUL SECONDO ESEMPIO, PREDETTO COME VIRGINICA MA E' IN REALTA' VERSICOLOR

Print the index of the test instances and the corresponding predictions

In [ ]:
# print the corresponding instances indexes and class names 
for i in range(len(iris_y_test)): 
    print("Instance # "+str(indices_test[i])+": ")
    print("Predicted: "+iris.target_names[predicted_y_test[i]]+"\t True: "+iris.target_names[iris_y_test[i]]+" ["+
                        ("OK" if iris.target_names[predicted_y_test[i]]==iris.target_names[iris_y_test[i]] else "NOT OK")
          +"]\n")

# Look at the specific examples (TEST SET)

In [ ]:
for i in range(len(iris_y_test)): 
    print("Instance # "+str(indices_test)+": ")
    s=""
    for j in range(len(iris.feature_names)):
        s=s+iris.feature_names[j]+"="+str(iris_X_test[i][j])
        if (j<len(iris.feature_names)-1): s=s+", "
    print(s)
    print("Predicted: "+iris.target_names[predicted_y_test[i]]+"\t True: "+iris.target_names[iris_y_test[i]]+"                                              \t["+
                        ("OK" if iris.target_names[predicted_y_test[i]]==iris.target_names[iris_y_test[i]] else "NOT OK")
          +"]\n")

# Obtain model performance results

In [ ]:
# print some metrics results
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
acc_score = accuracy_score(iris_y_test, predicted_y_test)
print("Accuracy score: "+ str(acc_score))
AVG = 'macro'
f1=f1_score(iris_y_test, predicted_y_test, average=AVG) #F1 SCORE = BALANCED F-SCORE (MEDIA ARMONICA) = 2 * (precision * recall) / (precision + recal)
print("F1 score ("+AVG+"): "+str(f1))                                 
#'macro' = MEDIA NON PESATA DELLO SCORE SU OGNI CLASSE (OVVERO: ESEMPI PESATI TUTTI ALLO STESSO MODO)  --> MEGLIO, SOPRATTUTTO SE LE DISTRIBUZIONI SONO DIFFERENTI!
#'micro' = MEDIA CALCOLATA GLOBALMENTE SUI TP, FN, FP

# Use Cross Validation (5 FOLDS)

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score # will be used to separate training and test
iris = load_iris()
clf = tree.DecisionTreeClassifier(criterion="entropy",random_state=300,min_samples_leaf=5,class_weight={0:1,1:1,2:1})
clf = clf.fit(iris.data, iris.target)
scores = cross_val_score(clf, iris.data, iris.target, cv=5) # score will be the accuracy
print(scores)

In [ ]:
# computes F1-score
f1_scores = cross_val_score(clf, iris.data, iris.target, cv=5, scoring='f1_'+AVG)
print(AVG)
print(f1_scores)

# Show the resulting tree 

## 2. Generate a picture here

In [ ]:
print(list(iris.feature_names))
print(list(iris.target_names))

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=iris.feature_names, 
                         class_names=iris.target_names, 
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)  
graph

#NODI BIANCHI: ENTROPIA MOLTO ALTA (NON HOMOGENEOUS)
#NODI ARANCIONI: SETOSA
#NODI VERDI: VERSICOLOR
#NODI VIOLA: VIRGINICA

#IL COLORE E' PIU DENSO --> L'ENTROPIA E' MINORE

# Your work: what you have to do
Modify the given Jupyter notebook on decision trees on Iris data and perform the following tasks:

1. get an artificial inflation of some class in the training set by a given factor: 10 (weigh more the classes virginica e versicolor which are more difficult to discriminate). Learn the tree in these conditions.
    1.b) modify the weight of some classes (set to 10 the weights for misclassification between virginica into versicolor and vice              versa) and learn the tree in these conditions. You should obtain similar results as for step 1.
2. learn trees but try to avoid overfitting (by improving the error on the test set) tuning the hyper-parameters on: the minimum number of samples per leaf, max depth of the tree, min_impurity_decrease parameters, max leaf nodes, etc.
3. build the confusion matrix of the created tree models on the test set and show them. 
4. build the ROC curves (or coverage curves in coverage space) and plot them for each tree model you have created: for each model you have to build three curves, one for each class, considered in turn as the positive class.

### 1. ARTIFICIAL INFLATION

In [ ]:
TESTSET_SIZE = 20
INFLATION_FACTOR = 20   #1A
WEIGHT_FACTOR = 20      #1B
AVG = 'macro'

In [ ]:
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# creo il classificatore (1a)
clf_1a = tree.DecisionTreeClassifier(criterion="entropy",random_state=300,min_samples_leaf=5,class_weight={0:1,1:1,2:1}) 
# creo il classificatore (1b) coi pesi ricalibrati per versicolor, virginica
clf_1b = tree.DecisionTreeClassifier(criterion="entropy",random_state=300,min_samples_leaf=5,class_weight={0:1,1:WEIGHT_FACTOR,2:WEIGHT_FACTOR}) 
                                                                                                        #0:setosa, 1:versicolor, 2:virginica

# creo il dataset
iris = load_iris()
X = iris.data
y = iris.target
for i, n in enumerate(iris.target_names):
    print(i, "-->", n)

# divido in trainingset e testset con random permutation, (1a) effettuo art.inflation sul training set x10 per versicolor, virginica
np.random.seed(0)
indices = np.random.permutation(len(X))
indices_train = indices[:-TESTSET_SIZE]
indices_test = indices[-TESTSET_SIZE:]
X_train = X[indices_train]
y_train = y[indices_train]
X_test  = X[indices_test]
y_test  = y[indices_test]
X_train_inflated = X_train
y_train_inflated = y_train
for i in range(len(X_train)):
    if y_train[i] == 1 or y_train[i] == 2:
        for j in range(INFLATION_FACTOR):
            X_train_inflated = np.append(X_train_inflated, [X_train[i]], 0)
            y_train_inflated = np.append(y_train_inflated, [y_train[i]], 0)
print("TRAINING SET SHAPE: X", X_train.shape, "y", y_train.shape)
print("TRAINING SET SHAPE (WITH INFLATION x"+str(INFLATION_FACTOR)+"): X", X_train_inflated.shape, "y", y_train_inflated.shape)
print("TEST SET SIZE:", TESTSET_SIZE)

# apprendimento su trainingset
clf_1a.fit(X_train_inflated, y_train_inflated)
clf_1b.fit(X_train, y_train)

# evaluation su testset
y_test_pred_1a = clf_1a.predict(X_test)
y_test_pred_1b = clf_1b.predict(X_test)
print("Predictions on test set:")
print(y_test_pred_1a, "(1A)")
print(y_test_pred_1b, "(1B)")
print("True classes:")
print(y_test) 
print("Errors (1A):")
print(np.array([0 if a==b else 1 for (a,b) in list(zip(y_test_pred_1a,y_test))]))
print("Errors (1B):")
print(np.array([0 if a==b else 1 for (a,b) in list(zip(y_test_pred_1b,y_test))]))

# accuracy e f1 score
acc_score_1a = accuracy_score(y_test, y_test_pred_1a)
acc_score_1b = accuracy_score(y_test, y_test_pred_1b)
print("Accuracy score (1A): ", acc_score_1a)
print("Accuracy score (1B): ", acc_score_1b)
f1_score_1a = f1_score(y_test, y_test_pred_1a, average=AVG)
f1_score_1b = f1_score(y_test, y_test_pred_1b, average=AVG)
print("F1 score (1A) ("+AVG+"): ", f1_score_1a)
print("F1 score (1B) ("+AVG+"): ", f1_score_1b)

### 2,3. TUNE HYPER PARAMETERS TO PRODUCE DIFFERENT CLFs, BUILD CONFUSION MATRIXes

In [ ]:
WEIGHT_FACTOR_SETOSA = 1
WEIGHT_FACTOR_VERSICOLOR = 10
WEIGHT_FACTOR_VIRGINICA = 10
MIN_SAMPLES_LEAF = 5
MAX_LEAVES = 10
MAX_DEPTH = 10
MIN_IMPURITY = 0
CRITERION = 'entropy' #'gini'
AVG = 'macro'

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

# confusion matrix function
def draw_confusion_matrix(class_names,y_test,y_test_pred,clf_label):
    cf = confusion_matrix(y_test,y_test_pred)
    plt.figure()
    plt.imshow(cf, cmap=plt.cm.Blues)
    plt.title("Confusion Matrix \""+clf_label+"\"")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names)
    plt.yticks(tick_marks, class_names)
    thresh = cf.max() / 2.0
    for i, j in itertools.product(range(cf.shape[0]),range(cf.shape[1])):
        plt.text(j, i, cf[i, j],horizontalalignment="center",color="white" if cf[i, j] > thresh else "black",)
    plt.colorbar()
    plt.show()

#tune hyper parameters to produce different clf
clf_2 = tree.DecisionTreeClassifier(criterion=CRITERION,random_state=0,min_samples_leaf=MIN_SAMPLES_LEAF,max_depth=MAX_DEPTH,class_weight={0:WEIGHT_FACTOR_SETOSA,1:WEIGHT_FACTOR_VERSICOLOR,2:WEIGHT_FACTOR_VIRGINICA},max_leaf_nodes=MAX_LEAVES,min_impurity_decrease=MIN_IMPURITY) 

# apprendimento su trainingset
clf_2.fit(X_train, y_train)

# evaluation su testset
y_test_pred_2 = clf_2.predict(X_test)
print("Predictions on test set:")
print(y_test_pred_2, "(2)")
print("True classes:")
print(y_test)
print("Errors (2):")
print(np.array([0 if a==b else 1 for (a,b) in list(zip(y_test_pred_2,y_test))]))

# accuracy e f1 score
acc_score_2 = accuracy_score(y_test, y_test_pred_2)
print("Accuracy score (2): ", acc_score_2)
f1_score_2 = f1_score(y_test, y_test_pred_2, average=AVG)
print("F1 score (2) ("+AVG+"): ", f1_score_2)

# draw matrixes of all the models
class_names = iris.target_names
draw_confusion_matrix(class_names,y_test,y_test_pred_1a,"1A")
draw_confusion_matrix(class_names,y_test,y_test_pred_1b,"1B")
draw_confusion_matrix(class_names,y_test,y_test_pred_2,"2")


### 4. BUILD ROC CURVEs

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize

def bin_convert(y,positive_label):
    return np.where(y==positive_label,1,0)

def draw_3_roc_curves(y_test_setosa,predicted_y_test_setosa,y_test_versicolor,predicted_y_test_versicolor,y_test_virginica,predicted_y_test_virginica,name):
    plt.figure()
    fpr, tpr, _ = roc_curve(y_test_setosa, predicted_y_test_setosa)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label="Setosa (area = %0.2f)" % roc_auc)
    fpr, tpr, _ = roc_curve(y_test_versicolor, predicted_y_test_versicolor)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label="Versicolor (area = %0.2f)" % roc_auc)
    fpr, tpr, _ = roc_curve(y_test_virginica, predicted_y_test_virginica)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label="Virginica (area = %0.2f)" % roc_auc)
    plt.plot([0, 1], [0, 1], "k")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC clf "+name)
    plt.legend(loc="lower right")
    plt.show()

def build_roc_plot_3_classes_binary(y_test,predicted_y_test,name):
    y_test_setosa = bin_convert(y_test,0)
    predicted_y_test_setosa = bin_convert(predicted_y_test,0)
    y_test_versicolor = bin_convert(y_test,1)
    predicted_y_test_versicolor = bin_convert(predicted_y_test,1)
    y_test_virginica = bin_convert(y_test,2)
    predicted_y_test_virginica = bin_convert(predicted_y_test,2)

    print("Predictions on test set:","\t\t", predicted_y_test,"("+name+")")
    print("True classes:\t\t\t\t", y_test)
    print("Errors ("+name+"):\t\t\t\t", np.array([0 if a==b else 1 for (a,b) in list(zip(predicted_y_test,y_test))]))
    print()
    print()
    print("Predictions on test set - SETOSA",":\t", predicted_y_test_setosa,"("+name+")")
    print("True classes - SETOSA:\t\t\t", y_test_setosa)
    print("Errors ("+name+"):\t\t\t\t", np.array([0 if a==b else 1 for (a,b) in list(zip(predicted_y_test_setosa,y_test_setosa))]))
    print()
    print("Predictions on test set - VERSICOLOR",":\t", predicted_y_test_versicolor,"("+name+")")
    print("True classes - VERSICOLOR:\t\t", y_test_versicolor)
    print("Errors ("+name+"):\t\t\t\t", np.array([0 if a==b else 1 for (a,b) in list(zip(predicted_y_test_versicolor,y_test_versicolor))]))
    print()
    print("Predictions on test set - VIRGINICA",":\t", predicted_y_test_virginica,"("+name+")")
    print("True classes - VIRGINICA:\t\t", y_test_virginica)
    print("Errors ("+name+"):\t\t\t\t", np.array([0 if a==b else 1 for (a,b) in list(zip(predicted_y_test_virginica,y_test_virginica))]))

    draw_3_roc_curves(y_test_setosa,predicted_y_test_setosa,y_test_versicolor,predicted_y_test_versicolor,y_test_virginica,predicted_y_test_virginica,name)

build_roc_plot_3_classes_binary(y_test,y_test_pred_1a,"1A")
build_roc_plot_3_classes_binary(y_test,y_test_pred_1b,"1B")
build_roc_plot_3_classes_binary(y_test,y_test_pred_2,"2")